## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob

import numpy as np
import scipy as sp
from scipy import signal as sg
import scipy.io


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    power = [2, 4, 2]
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidences = RunPulseRateAlgorithm(data_fl, ref_fl, power)
        errs.append(errors)
        confs.append(confidences)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def bandpass_filter(signal, fs = 125): 
    """
    Ensures that pulse rate is restricted between 40BPM (beats per minute) and 240BPM. 
    """
    b, a = sg.butter(3, (40/60.0, 240/60.0), btype='bandpass', fs=fs)
    return sg.filtfilt(b, a, signal)

def fourier_transform(signal, fs):
    """
    Calculate the fourier transfrom of given sequence.
    """ 
    n_samples = len(signal)
    freqs = np.fft.rfftfreq(n_samples, 1/fs)
    fft = np.abs(np.fft.rfft(signal, n_samples))
    return freqs, fft

def confidence(freqs, fft, freq, window_half_width=1):
    """
    Calculate confidence by Fourier transformes data inputs.
    """ 
    window = (freqs > freq - window_half_width) & (freqs < freq + window_half_width)  
    confidence = np.sum(fft[window]) / np.sum(fft)
    return confidence

def predict_heart_rate(acc, ppg, fs, window_length = 8, window_shift = 2, min_bandpass = 40/60.0, max_bandpass = 240/60.0):
    """
    Predict heart rate and calculate confidence using ppg and accelerometer data 
    """
    
    # Define window shapes with given values to function
    window_length = window_length * fs
    window_shift = window_shift * fs
    
    # Create empty lists to store calculations
    bpm_predictions = []
    confidences = []
    
    # Make calculations with shifting windows
    for origin in range(0, len(ppg) - window_length + 1, window_shift):
        
        # Estimation for current window
        ppg_window = ppg[origin:origin+window_length]
        acc_window = acc[origin:origin+window_length]
        
        # Fourier transform
        ppg_freqs, ppg_fft = fourier_transform(ppg_window, fs)
        acc_freqs, acc_fft = fourier_transform(acc_window, fs)
        
        # Remove unwanted frequencies 
        ppg_fft[ppg_freqs <= min_bandpass] = 0.0
        ppg_fft[ppg_freqs >= max_bandpass] = 0.0
        acc_fft[acc_freqs <= min_bandpass] = 0.0
        acc_fft[acc_freqs >= max_bandpass] = 0.0
    
        # Pick frequency with largerst FFT coefficient
        ppg_freq = ppg_freqs[np.argmax(ppg_fft, axis=0)]  
        acc_freq = acc_freqs[np.argmax(acc_fft, axis=0)]
        
        freq = ppg_freq
        conf = confidence(ppg_freqs, ppg_fft, ppg_freq)
        
        # If the dominant accelerometer frequency is the same as the PPG, estimated heart rate is biased.
        # Make new estimation sor second suitable frequency.
        if abs(ppg_freq - acc_freq) == 0:
            
            # Calculate frequency
            sec_freq = ppg_freqs[np.argsort(ppg_fft, axis=0)[-2]]
            sec_conf = confidence(ppg_freqs, ppg_fft, sec_freq)
            
            # If larger freq
            if sec_conf > conf:
                freq, conf = sec_freq, sec_conf
        
        bpm_predictions.append(freq * 60) 
        confidences.append(conf)
        
    return bpm_predictions, confidences

def RunPulseRateAlgorithm(data_fl, ref_fl, power):
     # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    fs = 125
    # Compute pulse rate estimates and estimation confidence.
    # Bandpass filter
    ppg  = bandpass_filter(ppg, fs)
    accx = bandpass_filter(accx, fs) 
    accy = bandpass_filter(accy, fs)
    accz = bandpass_filter(accz, fs)
    
    # Magnitude of acceleration as 3D
    acc = np.sqrt(accx**power[0] + accy**power[1] + accz**power[2])
    
    # Load ground truth as column vector and convert to row vector
    ground_truth = sp.io.loadmat(ref_fl)['BPM0'].reshape(-1) 
    
    # Calculate heart rate prediction, confidence and errors
    predictions, c = predict_heart_rate(acc, ppg, fs)
    errors = np.abs(np.subtract(predictions, ground_truth))
    
    # Return per-estimate mean absolute error and confidence
    errors, c = np.array(errors), np.array(c)
    return errors, c

In [2]:
Evaluate()

15.062028245222931

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

1. **Code Description**: 

This implementation estimates the pulse rate from the PPG signal and a 3-axis accelerometer. The ground truth pulse rate is taken from ECG signals. The code produces an output every 2 seconds. 

Function descriptions: 
* *LoadTroikaDataset* returns two objects: 
    * data_fls which contains a list of all files with a name starting with 'DATA_' in the directory
    * ref_fls which contains a list of all files with a name starting with 'REF_' in the directory
* *LoadTroikaDataFile* extracts the signal in the form of a numpy array with PPG and 3-axis accelerometer measurements.
* *Evaluate* is global function that loads the dataset, run the pulse rate algorithm (RunPulseRateAlgorithm) and aggregate the evaluation metrics. 
* *RunPulseRateAlgorithm* this function is the heart of the algorithm: 
    * It first ensures that the signal is band passed through a 40BPM to 240BPM filter. 
    * Next, characteristic frequencies for each signal are calculated in 8 second windows with 6 second overlap between adjacent windows. Characteristic frequencies for the PPG data are compared against the highest characteristic frequencies for each accelerometer axis per time window. This comparison determines the pulse rate frequency. When all frequencies extracted are similar to the accelerometer frequencies, the largest frequency is returned. 
    * Converts the frequency to beats per minute
    * Compute the confidence 
    * Finally, the absolute error between the determined pulse rate and the reference ECG pulse rate data are calculated. The errors and confidences are returned from this function
    
    
2. **Data Description** 

The Troika dataset is used to build the algorithm.

Two-channel PPG signals, 3-axis accelerometer signals, and one-channel ECG signals were simultaneously recorded. All signals were sampled at fs = 125Hz.

The dataset is collected on 12 different subjects. 

The signal data are contained in files named 'DATA_XX_TYPEXX'. Each file has 6 rows: Row1 ECG data, Rows 2 and 3 are the two channels of PPG; and Rows 4,5, and 6 are the x-, y-, and z-axis accelerometer measurements. The ground truth data is contained in files named 'REF_XX_TYPEXX'

3. **Algorithm Description** 

This method estimates pulse rate using data from Photoplethysmography (PPG) sensors worn on the user's wrist. PPG sensors send light into a user's wrist and measure the quantity of light that returns to the PPG's photodector. The quantity of blood cells in one region is the physiological aspect that a PPG monitors.

* When the heart contracts, blood fills the extremities and thus more blood cells are measured by the sensor.
* When the heart relaxes, blood leaves the extremities and fill the heart --> less blood cells are measured.

Moving the arm when walking or running may interfere with the measurements as the blood cell movement changes. 
The outputs of the algorithm are: 
* The estimated pulse rate frequency (in BPM) 
* The confidence score of that prediction.

Drawbacks of using this algorithm: 
* The output from this algorithm only takes into account arm movement noise that is periodic.
* It does not handle non-periodic movements captured by sensors.
* Does not account for other noise sources such as ambient light or shifts in sensor position.

The major issue with this algorithm is the amount of data it is trained on. Note that using machine learning algorithm such random forest regression may increase its performance. 

4. **Algorithm Performance** 


The performance was calculated by calculating the mean absolute error between the estimated pulse rate frequency and the groud truth extracted from REF files. This algorithm meets the project's requirement that the mean absolute error at 90% availability is less than 15 BPM on the test dataset.
![passed.png](passed.png)

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 

In [ ]:
!zip -r part1.zip datasets passed.png 